## Task 2: Is Rectangle?

The objective of this task is to verify if 4 positive integers can represents the length of the sides of a rectangle, by using quantum computing.

Approach #1: create a quantum circuit to compare two bitstrings, and use it to compare each pair of the given numbers. In such an approach, for each pair, I encode each bit of the bitstrings of the two numbers in a different qubit, and implement a quantum circuit which compares the content of the qubits. Since each qubit is in a base state, measurement will give correct output with probability 1.

Approach #2: building on top of approach #1, I use entanglement to compare two pairs of different numbers with the same circuit, at the same time. Therefore, the number of times the quantum circuit is created is halved, reducing the execution time.

### Approach #1


Let's import the libraries we need.

In [1]:
import numpy as np
import pylatexenc
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit import Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.circuit.classicalfunction import classical_function
from qiskit.circuit.classicalfunction.types import Int1

Now we define a quantum circuit which, given two qubits in a base state ( |0> or |1>), compares them and sets the output on an ancilla qubit.

In [2]:
"""
    Such a function returns a quantum circuit which compares the content of two qubits (in a base state).
    It does so by implementing the classical logical boolean function:
    c = a AND b or NOT a AND NOT b
    by using qiskit.circuit.classical_function. It can also be designed by hand, by using CCX gates. 
"""

@classical_function
def qubit_comparator(a: Int1, b: Int1) -> Int1:
    return ((a and b) or (not a and not b))

In [3]:
print(qubit_comparator.synth())

               
q_0: ──■───────
       │       
q_1: ──┼────o──
     ┌─┴─┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘


As a next step, we define a quantum circuit which compares the content of two quantum registers, by using the previously defined comparator for each pair of qubits to compare. The method to generate the circuit also initializes the registers with two given bitstrings.

In [4]:
def register_comparator(N: int, value1: str, value2: str):
    """
        Such a function implements a quantum circuit which compares two registers of N qubits.
        Parameters:
            N : number of qubits
            value1 : bitstring to initialize first quantum register
            value2 : bitstring to initialize second quantum register
        Returns quantum circuit for bistrings comparison
    """
    assert N > 0
    assert len(value1) == N
    assert len(value1) == len(value2)
    
    circuit = QuantumCircuit(2*N+N, N)
    qubit_comp = qubit_comparator.synth()
    
    circuit.initialize(Statevector.from_label(value1), range(N))
    circuit.initialize(Statevector.from_label(value2), range(N,2*N))
    for i in range(N): 
        circuit.append(qubit_comp, [i, i+N, i + 2*N])
        circuit.measure(i + 2*N, i)
    return circuit

Then we define a function which receives as input a pair of two numbers, and, by using a quantum circuit, gives as output a Boolean representing whether the given numbers are equal or not. This method creates an hardware abstraction layer, by hiding the usage of quantum computing, and can be used by any user, even if not expert in quantum computing.

In [5]:
def check_numbers(value1: int, value2: int) -> bool:
    """
        This function receives as input two positive integers and it verifies if they are equal, by using a quantum circuit.
        Parameters:
            value1 : positive integer
            value2 : positive integer
        Returns a Boolean set to True if value1 equals value2, False otherwise.
    """
    assert value1 > 0
    assert value2 > 0
    
    bw1 = np.ceil(np.log2(value1))+1
    bw2 = np.ceil(np.log2(value2))+1
    bitwidth = int(max(bw1, bw2))
    
    format_string = '{0:0' + str(bitwidth) + 'b}'
    str1 = format_string.format(value1)
    str2 = format_string.format(value2)
    #print(bitwidth,str1, str2)
    quantum_circuit = register_comparator(bitwidth, str1, str2)
    print("QUANTUM CIRCUIT TO CHECK %d and %d:" % (value1, value2))
    print(quantum_circuit)
    # use local simulator
    aer_sim = Aer.get_backend('aer_simulator')
    shots = 1 #we get correct output with probability 1, therefore only 1 shot is required
    quantum_circuit = transpile(quantum_circuit, backend=aer_sim)
    qobj = assemble(quantum_circuit, shots=shots)
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    return "1" * bitwidth in counts
    

Finally, this method checks whether the given numbers can represent a rectangle, by verifying all possible pairs.

In [6]:
def is_rectangle (A: int, B: int, C: int, D: int):
    """
        A : integer value that is one side of the rectangle.
        B : integer value that is one side of the rectangle.
        C : integer value that is one side of the rectangle.
        D : integer value that is one side of the rectangle.
        Return if is a rectangle returns 1 else 0. 
    """
    assert A > 0 and B > 0 and C > 0 and D > 0
    
    AisB = check_numbers(A,B)
    if AisB and check_numbers(C,D):
        return 1
    AisC = check_numbers(A,C)
    if AisC and check_numbers(B,D):
        return 1
    AisD = check_numbers(A,D)
    if AisD and check_numbers(B,C):
        return 1
    return 0


As a final point, we test the function.

In [7]:
result = is_rectangle(8,8,10,10)

QUANTUM CIRCUIT TO CHECK 8 and 8:
      ┌──────────────────────────────────────────────┐┌───────────────────┐»
 q_0: ┤0                                             ├┤0                  ├»
      │                                              ││                   │»
 q_1: ┤1                                             ├┤                   ├»
      │  Initialize(0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0) ││                   │»
 q_2: ┤2                                             ├┤                   ├»
      │                                              ││                   │»
 q_3: ┤3                                             ├┤                   ├»
      ├──────────────────────────────────────────────┤│                   │»
 q_4: ┤0                                             ├┤1 qubit_comparator ├»
      │                                              ││                   │»
 q_5: ┤1                                             ├┤                   ├»
      │  Initialize(0,0,0,0,0,0,0,0,1,0,0,

In [8]:
print("The output is:", result)

The output is: 1


## Approach #2: Entanglement

Starting from Approach #1, we can speedup computation by using the same circuit to compare TWO couples at a time, instead of 1, at the only expense of 1 additional qubit. We can do so by entangling the two pairs of numbers.

We define a new register comparator, in which input is given as numbers (not bitstring anymore). <br>
This quantum circuit will have 1 additional qubit, and the qubits will be initialized in an entangled state, in such a way that, when 0 on the ancilla qubit is measured, the comparison of the first pair is obtained, while when 1 is measured, the comparison for the second pair is measured.

In [9]:
def register_comparator_2(N: int, value1: int, value2: int, value3: int, value4: int):
    """
        Such a function implements a quantum circuit which compares two pairs of numbers, in entanglement.
        Parameters:
            N : number of qubits
            value1 : number to compare against value2
            value2 : number to compare against value1
            value3 : number to compare against value4
            value4 : number to compare against value3
        Returns quantum circuit for number comparison; when 0 is measured on first qubit, output is comparison between value1 and value2;
        when 1 is measured on first qubit, output is comparison between value3 and value4
    """
    assert N > 0
    assert np.ceil(np.log2(value1)) <= N
    assert np.ceil(np.log2(value2)) <= N
    
    circuit = QuantumCircuit(2*N+N+1, N+1)
    qubit_comp = qubit_comparator.synth().decompose()
    
    state = np.zeros(2 ** (2 * N + 1))
    state[value1 * (2 ** N) + value2 ] = 1/np.sqrt(2)
    state[2 ** (2 * N) + value3 * (2 ** N) + value4 ] = 1/np.sqrt(2)
    circuit.initialize(state, range(2*N+1))
    for i in range(0,N): 
        circuit.append(qubit_comp, [i, i+N, i+2*N+1])
    circuit.measure(range(2*N+1,3*N+1), range(0, N))
    circuit.measure(2*N,N)
    return circuit

Now we introduce a function to check numbers: it reates an hardware abstraction layer with respect to the quantum circuit, in order to allow user to compare numbers without explicitly using qiskit library. Such a function compares two pairs of numbers.

In [10]:
def check_numbers_2(value1: int, value2: int, value3: int, value4: int) -> bool:
    """
        This function receives as input two pairs of positive integers and it verifies if they are equal, by using a quantum circuit.
        Parameters:
            value1 : number to compare against value2
            value2 : number to compare against value1
            value3 : number to compare against value4
            value4 : number to compare against value3
        Returns a Boolean set to True if value1 equals value2 and value3 equals value4, False otherwise.
    """
    assert value1 > 0
    assert value2 > 0
    
    bw1 = np.ceil(np.log2(value1))+1
    bw2 = np.ceil(np.log2(value2))+1
    bw3 = np.ceil(np.log2(value3))+1
    bw4 = np.ceil(np.log2(value4))+1
    bitwidth = int(max(bw1, bw2, bw3, bw4))
    
    quantum_circuit = register_comparator_2(bitwidth, value1, value2, value3, value4)
    print("QUANTUM CIRCUIT TO CHECK %d with %d and %d with %d:" % (value1, value2, value3, value4))
    print(quantum_circuit)
    # use local simulator
    aer_sim = Aer.get_backend('aer_simulator')
    shots = 10 #we get correct output with probability 1, therefore only 1 shot is required
    quantum_circuit = transpile(quantum_circuit, backend=aer_sim)
    qobj = assemble(quantum_circuit, shots=shots)
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    return "1"+"1" * bitwidth in counts and "0"+"1" * bitwidth in counts
    

Finally, we define a method to verify if the given numbers can be sides of a rectangle.

In [11]:
def is_rectangle_2 (A: int, B: int, C: int, D: int):
    """
        A : integer value that is one side of the rectangle.
        B : integer value that is one side of the rectangle.
        C : integer value that is one side of the rectangle.
        D : integer value that is one side of the rectangle.
        Return if is a rectangle returns 1 else 0. 
    """
    assert A > 0 and B > 0 and C > 0 and D > 0
    
    if check_numbers_2(A,B,C,D):
        return 1
    if check_numbers_2(A,C,B,D):
        return 1
    if check_numbers_2(A,D,B,C):
        return 1
    else:
        return 0

In [12]:
result = is_rectangle_2(8,10,10,8)

QUANTUM CIRCUIT TO CHECK 8 with 10 and 10 with 8:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
 c: 6/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

QUANTUM CIRCUIT TO CHECK 8 with 8 and 10 with 10:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
 c: 6/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [13]:
print("The output is:", result)


The output is: 1


## Final comparison

To compare the two solution, we verify the execution time of both calls, by considering the worst case: given inputs A,B,C,D, A == B == C but A != D (which implies 5 quantum circuits to execute on approach #1 and 3 on approach #2).

In [14]:
%%time
is_rectangle(180,180,180,190)

QUANTUM CIRCUIT TO CHECK 180 and 180:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
q_18: »
      »
q_19: »
      »
q_20: »
      »
q_21: »
      »
q_22: »
      »
q_23: »
      »
q_24: »
      »
q_25: »
      »
q_26: »
      »
 c: 9/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

QUANTUM CIRCUIT TO CHECK 180 and 190:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
q_18: »
      »
q_19: »
      »
q_20: »
      »
q_21: »
      »
q_22: »
      »
q_23: »
      »
q_24: »
      »
q_25: »
      »
q_26: »
      »
 c: 9/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

QUANTUM CIRCUIT TO CHECK 180 and 180:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
q_18: »
      »
q_19: »
      »
q_20: »
      »
q_21: »
      »
q_22: »
      »
q_23: »
      »
q_24: »
      »
q_25: »
      »
q_26: »
      »
 c: 9/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

QUANTUM CIRCUIT TO CHECK 180 and 190:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
q_18: »
      »
q_19: »
      »
q_20: »
      »
q_21: »
      »
q_22: »
      »
q_23: »
      »
q_24: »
      »
q_25: »
      »
q_26: »
      »
 c: 9/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

QUANTUM CIRCUIT TO CHECK 180 and 190:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
q_18: »
      »
q_19: »
      »
q_20: »
      »
q_21: »
      »
q_22: »
      »
q_23: »
      »
q_24: »
      »
q_25: »
      »
q_26: »
      »
 c: 9/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

CPU times: user 8min 5s, sys: 6.89 s, total: 8min 12s
Wall time: 1min 3s


0

In [15]:
%%time
is_rectangle_2(180,180,180,190)

QUANTUM CIRCUIT TO CHECK 180 with 180 and 180 with 190:


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



QUANTUM CIRCUIT TO CHECK 180 with 180 and 180 with 190:


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



QUANTUM CIRCUIT TO CHECK 180 with 190 and 180 with 180:


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 6min 52s, sys: 13.3 s, total: 7min 5s
Wall time: 1min 59s


0

As you can see, Approach #2 results to be faster than Approach #1 (6min 52s versus 8min 5s, on an Intel i7-7700HQ), due to the higher number of circuits to run required by Approach #1. Yet, I experienced cases in which Approach #1 resulted to be faster than approach #2. Why? The answer is simple: even though the number of circuits to create and run is higher for approach #1, the number of qubits used, which is logarithmic w.r.t the maximum number of the inputs, is lower for approach #1 (because of the ancilla qubit used by approach #2), and we know that, for emulators, an increase in the number of qubits implies an exponential increase in the execution time. Additionally, Approach #2 requires more shots to run, while Approach #1 only requires 1 shot, since it gives the correct output with probability = 1. Unfortunately, I cannot run such circuits on a real quantum hardware, so I can finally say that, for what concerns quantum emulators, Approach #2 generally wins, but the real performance effectively depends on the instance of the problem.